# Receptor-ligand interaction heatmaps
Previously to running this, you need to have a ./freqs directory with subdirectories for receptor-ligand interaction (hydrogen bonds: "hb", water bridges, "wb", van der Waals: "vdw"). Inside, there should be GetContacts frequency files for each ligand (A, B...) and replica (1, 2...), like:



## 0. Packages

In [5]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

## 1. Prepare data

In [6]:
rec_lig_df = pd.DataFrame({ "lig": [],     "rep": [], "simulation": [],
                             "aa": [],    "freq": [], "type": []})

for lig in []: # Ligands
    for int_type in ["hb", "wb", "vdw"]: # Hydrogen bond, water bridge, Van der Waals
        for rep in []: # Replicas

            # df is a temporary data frame to fill the main data frame
            # lig_id: e.g. L:LIG:1 is the id given by GetContacts to the ligand
           
            df = pd.read_csv("freqs/" + int_type + "/" + lig + rep + "_freqs.tsv", 
                             sep = "\t", comment = "#", header = None, names = ["lig_id", "aa", "freq"]) 
            
            df.drop(["lig_id"], axis=1, inplace=True) # drop lig_id
            
            df["lig"] = lig
            df["rep"] = rep
            df["simulation"] = lig + rep
            df["type"] = int_type

            rec_lig_df = pd.concat([rec_lig_df, df], ignore_index = True)

rec_lig_df

,lig,rep,simulation,aa,freq,type


In [7]:
# Include 0 contact frequencies
for lig in ["G"]:
    for aa in set(rec_lig_df["aa"]):
        for int_type in ["hb", "wb", "vdw"]:
            df = rec_lig_df.loc[(rec_lig_df["lig"] == lig) & 
                                 (rec_lig_df["rep"] == rep) & 
                                 (rec_lig_df["aa"] == aa) &
                                 (rec_lig_df["type"] == int_type)]
            
            # If there is not a frequency, create a temporary dict, convert it to data frame
            # and append it to the frequencies data frame
            if df.empty:
                
                dict_to_df = {"aa": [aa],
                              "freq": [0], 
                              "lig": [lig],
                              "rep": [rep],
                              "simulation": [lig + rep], 
                              "type": [int_type]} 
                
                df = pd.DataFrame(dict_to_df)
                rec_lig_df = pd.concat([rec_lig_df, df], ignore_index = True)

rec_lig_df

,lig,rep,simulation,aa,freq,type


In [8]:
# Select orthosteric binding site (OBS) 
# e.g. obs = ["P:ASP:114", "P:VAL:115", "P:ILE:183", 
#             "P:ILE:184", "P:SER:193", "P:SER:194", 
#             "P:SER:197", "P:PHE:389", "P:PHE:390",
#             "P:HSD:393"]
obs = []

rec_lig_df = rec_lig_df[rec_lig_df["aa"].isin(obs)]
rec_lig_df

,lig,rep,simulation,aa,freq,type


## 2. Plot heatmaps

In [9]:
# Chat GPT code
# Define the desired order of the columns
col_order = []

# Define a function to create heatmaps for each type value
def create_heatmap(df, type_value):
    # Subset the data frame for the current type value
    subset_df = df[df['type'] == type_value]

    # Reorder the rows by ligand name
    subset_df['lig'] = pd.Categorical(subset_df['lig'], 
                                      categories=[], # Write here the ligand names
                                      ordered=True)

    # Sort the aa column by the desired order
    subset_df['aa'] = pd.Categorical(subset_df['aa'], categories=col_order, ordered=True)
    subset_df = subset_df.sort_values('aa')

    # Pivot the data to create the heatmap
    pivoted_df = subset_df.pivot_table(index=['lig'], columns=['aa'], values='freq', aggfunc='mean')

    # Create the heatmap using seaborn
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.heatmap(pivoted_df, cmap='Purples', annot=True, fmt='.2f', ax=ax, vmin=0, vmax=1)
    ax.set_title(type_value)
    ax.set_xlabel('OBS residue')
    ax.set_ylabel('Ligand')
    
    # Rotate the y-axis tick labels to be vertical
    ax.tick_params(axis='y', labelrotation=90)

In [10]:
# Font size
sns.set(font_scale = 1.28)

# Group the data frame by the "type" column and iterate over each group
for type_value, group_df in rec_lig_df.groupby('type'):
    create_heatmap(group_df, type_value)